In [1]:
# Needed Libraries
!pip install tensorflow
!pip install tflearn
import nltk
from nltk.stem import SnowballStemmer
import pandas as pd
import numpy as np
import tflearn
import tensorflow as tf
import random

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.3/107.3 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for tflearn: filename=tflearn-0.5.0-py3-none-any.whl size=127283 sha256=67d001184a6e77fdcf5689c455d76d997f59c51a872f33289f9f06d018b8a2b7
  Stored in directory: /root/.cache/pip/wheels/55/fb/7b/e06204a0ceefa45443930b9a250cb5ebe31def0e4e8245a465
Successfully built tflearn


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
# Reset the tflearn graph from earlier training
from tensorflow.python.framework import ops
ops.reset_default_graph()

In [3]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
stemmer = SnowballStemmer('english')

In [5]:
# Set the directory path for loading the datasets
%cd /content/drive/MyDrive/ELM

/content/drive/MyDrive/ELM


In [6]:
# Load the Trilingual Dataset
import json
with open('intents_eng_update.json', 'r') as json_data:
  intents = json.load(json_data)

# Load yoruba dataset
with open('intent_yoruba_update.json', 'r') as json_data:
  intents_yo = json.load(json_data)

# Load hindi dataset
with open('intent_hindi_update.json', 'r') as json_data:
  intents_hi = json.load(json_data)

In [ ]:
intents

In [7]:
# After loading the JSON file, begin collect of the three langauge into documents, words and intent class.

documents = []
classes = []
words  = []
ignore_words = ['?']

#loop through each sentence in our intent file pattern in english dataset
for intent in intents['intents']:
  for pattern in intent['patterns']:
    #tokenize each word in the pattern
    w = nltk.word_tokenize(pattern)
    #add to our words list
    words.extend(w)
    #add to document in our corpus
    documents.append((w, intent['tag']))
    #add to our classes list
    if intent['tag'] not in classes:
      classes.append(intent['tag'])


#loop through each sentence in our intent file pattern yoruba dataset
for intent in intents_yo['intents']:
  for pattern in intent['patterns']:
    #tokenize each word in the pattern
    w = nltk.word_tokenize(pattern)
    #add to our words list
    words.extend(w)
    #add to document in our corpus
    documents.append((w, intent['tag']))
    #add to our classes list
    if intent['tag'] not in classes:
      classes.append(intent['tag'])


#loop through each sentence in our intent file pattern hindi dataset
for intent in intents_hi['intents']:
  for pattern in intent['patterns']:
    #tokenize each word in the pattern
    w = nltk.word_tokenize(pattern)
    #add to our words list
    words.extend(w)
    #add to document in our corpus
    documents.append((w, intent['tag']))
    #add to our classes list
    if intent['tag'] not in classes:
      classes.append(intent['tag'])

#stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

#remove duplicates
classes = sorted(list(set(classes)))

print(len(documents), 'documents')
print(len(classes), 'classes', classes)
print(len(words), 'unique stemmed words', words)


143 documents
44 classes ['O dabọ', 'alavida', 'apanilẹrin', 'awọn nkan', 'bhugataan', 'delivery', 'funny', 'goodbye', 'greeting', 'idupẹ', 'ifijiṣẹ', 'igbaradi akoko', 'ikini', 'iru ile', 'items', 'majedaar', 'mittee ke prakaar', 'nutrition', 'orisi', 'ounje', 'owo sisan', 'payments', 'poshan', 'prakaar', 'preparation time', 'saamaan', 'shubhakaamana', 'shukriya', 'soil type', 'taiyaaree ka samay', 'thanks', 'types', 'vitaran', 'अलविदा', 'तैयारी का समय', 'पोषण', 'प्रकार', 'भुगतान', 'मजेदार', 'मिट्टी के प्रकार', 'वितरण', 'शुक्रिया', 'शुभकामना', 'सामान']
335 unique stemmed words ['!', "'s", 'a', 'aadarsh', 'aap', 'aapak', 'aapako', 'abhe', 'abhivaadan', 'accept', 'achchha', 'achchhe', 'akoko', 'alavida', 'anfani', 'anyon', 'apanilẹrin', 'ara', 'are', 'awada', 'awọn', 'baad', 'baay', 'bahut-bahut', 'banaan', 'bataen', 'batao', 'bawo', 'bechat', 'benefit', 'best', 'bhugataan', 'bodi', 'bye', 'can', 'card', 'cash', 'chaay', 'chutakula', 'coffe', 'credit', 'dabọ', 'dada', 'dara', 'day', 'de

In [8]:
#creating our training data
training = []
output = []
#create an empty array for our output
output_empty = [0] * len(classes)

#training set, bag of words for each sentence
for doc in documents:
  #initialize our bag of words
  bag = []
  #list of tokenized words for the pattern
  pattern_words = doc[0]
  # stem each word
  pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
  # create our bag od words array
  for w in words:
    bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists
train_x = list(training[:,0])
train_y = list(training[:,1])


<ipython-input-8-9344e34e6ec6>:27: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [9]:
# Split into train and test sets
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(train_x, train_y, test_size=0.20, random_state=10, stratify=train_y)

In [ ]:
train_x

In [10]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [11]:
import time

# Start the timer
start_time = time.time()

# Your model training code here

In [12]:
# reset underlying graph data
tf.reset_default_graph()
# Build neural network

net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.embedding(net, input_dim=len(words), output_dim=10,  trainable=True, name="EmbeddingLayer")
# ELM Layer
net = tflearn.fully_connected(net, 128)
net = tflearn.activations.tanh(net)

# create attention layer
attention_probs = tflearn.fully_connected(net, 1, activation='softmax')
attention_mul = tf.multiply(net, attention_probs)

# add fully connected layer and output layer
net = tflearn.fully_connected(attention_mul, len(train_y[0]), activation='softmax')
#net = tflearn.regression(net)
net = tflearn.regression(net, optimizer='adam', loss='categorical_crossentropy', metric='default', learning_rate=0.001)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
# Start training (apply gradient descent algorithm)
hist=model.fit(train_x, train_y, n_epoch=50, batch_size=64, show_metric=True, validation_set=(test_x, test_y))

model.save('model.tflearn')

Training Step: 29949  | total loss: 0.00002 | time: 8.236s
| Adam | epoch: 050 | loss: 0.00002 - acc: 1.0000 -- iter: 38272/38324
Training Step: 29950  | total loss: 0.00002 | time: 9.576s
| Adam | epoch: 050 | loss: 0.00002 - acc: 1.0000 | val_loss: 0.00002 - val_acc: 1.0000 -- iter: 38324/38324
--


In [13]:
# Stop the timer
end_time = time.time()

# Calculate the elapsed time
training_time = end_time - start_time
print("Training time: ", training_time, "seconds")

Training time:  626.1890141963959 seconds


In [ ]:
model.evaluate(test_x, test_y)

In [ ]:
predicted_x = model.predict(test_x)
#y_hatclass = model.predict_classes(test_x)
predicted_x


In [ ]:
import numpy
#METHOD 1
#accuracy = model.evaluate(test_x, test_y)

#METHOD 2
predictions = model.predict(test_x)
accuracy = 0
for prediction, actual in zip(predictions, test_y):
    predicted_class = numpy.argmax(prediction)
    actual_class = numpy.argmax(actual)
    if(predicted_class == actual_class):
        accuracy+=1
accuracy = accuracy / len(test_y)

In [ ]:
accuracy

In [ ]:
import sklearn.metrics
Y_pred = model.predict(test_x)
Y_pred_classes = np.argmax(Y_pred,axis = 1)

import numpy as np
test_y = np.argmax(test_y, axis=1)


In [ ]:
#calculate precision, recall and F1 score
import sklearn.metrics
sklearn.metrics.precision_recall_fscore_support(Y_pred_classes, test_y,  beta=1.0, labels=None, pos_label=1, average=None, warn_for=('precision', 'recall', 'f-score'), sample_weight=None, zero_division='warn')

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

confusion_mtx = sklearn.metrics.confusion_matrix(test_y, Y_pred_classes)
f,ax = plt.subplots(figsize=(8, 8))
sns.heatmap(confusion_mtx, annot=True, linewidths=0.01,cmap="Blues",linecolor="black", fmt= '.1f',ax=ax)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()

In [16]:
# Prepare user input for onward dialogue
def clean_up_sentence(sentence):
  #tokenize the pattern
  sentence_words = nltk.word_tokenize(sentence)
  # stem each word
  sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
  return sentence_words

# return bag of words array : 0 or 1 for each word in the bag that exit in the sentence
def bow(sentence, words, show_details=False):
  # tokenize the pattern
  sentence_words = clean_up_sentence(sentence)
  # bag of words
  bag = [0]*len(words)
  for s in sentence_words:
    for i,w in enumerate(words):
      if w == s:
        bag[i] = 1
        if show_details:
          print("found in bag: %s" %w)
  return(np.array(bag))


In [ ]:
# Dialouge section in which response to user inputs are retrieved based on a probabilty cut off

agent_name = 'Coffee chat'
ERROR_THRESHOLD = 0.75
def classify(sentence):
    # generate probabilities from the model
    results = model.predict([bow(sentence, words)])[0]
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list



In [29]:
classify('what  payment method do you accept')

[]

In [ ]:
# Dialogue module
def response(sentence, userID='123', show_details=False):
  results = classify(sentence)
  # if we have a classification then find the matching intent tag
  if results:
    # loop as long as there are matches to process
    while results:

      for i in intents['intents']:
        # find a tag matching the first result
        if i['tag'] == results[0][0]:
          # a random response from the intent
          return print(f"{agent_name}: {random.choice(i['responses'])}")

      for a in intents_yo['intents']:
        # find a tag matching the first result
        if a['tag'] == results[0][0]:
          # a random response from the intent
          return print(f"{agent_name}: {random.choice(a['responses'])}")
      for b in intents_hi['intents']:
        # find a tag matching the first result
        if b['tag'] == results[0][0]:
          # a random response from the intent
          return print(f"{agent_name}: {random.choice(b['responses'])}")

    results.pop(0)
  else:
      print(f'{agent_name} : I do not understand kindly rephrase your question...')

# Define quiting command in either of the languages
eng="quit"
yor="jáwọ́"
hind="छोड़ना"
hineng="chhodana"


print("Coffee Chat: Hi! How can I assist you today? \nif you wich to end the conversation type Quit or jáwọ́ or छोड़ना or chhodana")
while True:
    user_input = input("User: ")
    if user_input.lower() == eng:
        print(f"{agent_name}: Goodbye!")
        break
    if user_input.lower() == yor:
        print(f"{agent_name}: O dabọ o ṣeun fun àbẹwò")
        break
    if user_input.lower() == hineng:
        print(f"{agent_name}: alavida aane ke lie dhanyavaad")
        break
    if user_input.lower() == hind:
        print(f"{agent_name}: अलविदा आने के लिए धन्यवाद")
        break
    response(user_input, show_details=True)
